In [2]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [3]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [4]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [5]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [6]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [7]:
data.shape

(999, 5)

In [7]:
data.head(3)

,content,keywords,summary,title,url
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html
1,"Вчера «Газпром» снизил верхнюю планку прогноза собственной добычи газа в 2020 году. Через 12 лет концерн собирается добывать около 620–640 млрд. куб. м в год. При этом общее производство газа в стране, по расчетам холдинга, должно достичь 940 млрд. куб. м. Иными словами, треть добываемого объема, по мнению холдинга, должны будут обеспечить независимые производители. Эксперты не верят, что независимые компании смогут выйти на такие объемы добычи. Если расчеты «Газпрома» не оправдаются, то под ударом окажутся отечественные предприятия и население, которым придется сокращать потребление и смириться с новым витком цен. Иных путей покрытия возможного дефицита газа нет, так как вряд ли холдинг разорвет уже заключенные контракты на экспорт газа в другие страны. \n«Газпром» к 2020 году планирует добывать 620–640 млрд. куб. м газа, сообщил вчера на форуме «ТЭК России в ХХI веке» глава управления по добыче газа, газового конденсата и нефти холдинга Валерий Минликаев. Тем самым он уточнил пре...","[газпром, газ]",,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazprom.html
2,"Долголетний труд Евгения Витковского на ниве перевода, а также в качестве редактора и антологиста известен многим. Но не все знают его как поэта и прозаика. В этом году уже вышла составленная им и Еленой Кистеровой антология «Раздол туманов: Страницы шотландской гэльской поэзии XVII–XX вв.», а в апреле запланирован выход его романа «Протей, или Византийский кризис» (отрывок из романа читайте на с. 12). С \n побеседовал \n– Одна из таких книг только что вышла – «Раздол туманов. Страницы шотландской гэльской поэзии XVII–XX веков». Это стихи 29 поэтов, все в переводе с оригинала – моем и Елены Кистеровой. Работа заняла 10 лет, включая изучение языка. Она была упоительно интересной: до нас переводов из этой поэзии на русский не было вовсе. Сейчас должен выйти том стихотворений канадского классика Роберта Уильяма Сервиса, «канадского Киплинга», около 300 стихотворений. Кроме того, в Петербурге в производстве наш огромный трехтомный плод совместной работы – антология «Франция в сердце»....","[франсуа рабле, сервантес, шекспир, конан дойл, михаил булгаков, александр грин, борхес, босх, маркес, герман гессе, голландская живопись, гаргантюа и пантагрюэль, дон кихот, мастер и маргарита, москва, россия, история, поэзия, шотландия, баллада, пере]","Евгений Витковский о том, как Босх протягивает руку Шекспиру, \r\nи оба танцуют в пламени пожара в охваченном чумой средневековом городе",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitkovsky.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [13]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [14]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [10]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.06
F1 -  0.06
Jaccard -  0.03


Или 10.

In [11]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.07
F1 -  0.06
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [12]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [13]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [14]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [15]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                                      [в, и, на, не, что, –, его, «молодежное, с, это]
1                                                            [в, и, на, –, млрд., куб., по, к, газа, м]
2                                                                 [в, –, и, не, я, но, что, это, на, с]
3                                                       [в, на, и, ким, по, –, что, видео, он, зинаиды]
4                                              [в, и, на, новой, площадью, москвы, –, развития, с, для]
5                                                             [в, на, и, не, с, но, уже, что, у, гонки]
6                                                  [на, в, (с, место)., и, рф, позиции, влияние, по, с]
7                                                        [в, и, –, по, с, культуре, не, из, будет, как]
8                                                              [в, и, на, с, что, для, по, –, не, газа]
9    [в, рф, террористической, организации, задержаны, –, четвер

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [15]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [17]:
data['content_norm'] = data['content'].apply(normalize)

In [18]:
data['title_norm'] = data['title'].apply(normalize)

In [19]:
data['title_norm'].head(10)

0            [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                 [газпром, хватить]
2                                      [бесконечный, партия, четырехмерный, шахматы]
3    [экс-депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                 [новый, москва, остаться, территория, экологический, безопасность]
5                         [f1, гран-при, сша, пройти, четыре, машина, стопка, штраф]
6                                [100, ведущий, политик, россия, февраль, 2018, год]
7                                           [закон, культура, принимать, фон, арест]
8                     [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                       [фсб, калужский, область, задержать, четверо, участник, иго]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [20]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [21]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.14
F1 -  0.12
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [22]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                         [яблоко, молодёжный, который, акция, год, активист, это, деятельность, политика, наш]
1                               [миллиард, газа, год, куб, метр, газпром, добыча, 2020, должный, производитель]
2                                              [год, это, книга, роман, тот, писать, выйти, один, мир, перевод]
3                                    [ким, зинаида, видео, год, журналист, суд, дело, бывший, футиный, который]
4                         [площадь, территория, новый, москва, га, который, столица, тинао, парковый, развитие]
5                                  [гонка, который, команда, место, позиция, два, один, из-за, круг, чемпионат]
6                        [место, влияние, рф, позиция, глава, россия, президент, сергей, политический, рейтинг]
7                 [культура, закон, который, сфера, стд, разработать, концепция, проект, изменение, сообщество]
8                              [газопровод, сирия, год, турция, газа, россия, европа, катар, который, ту

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [23]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [24]:
data['content_norm'] = data['content'].apply(normalize)

In [25]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Ещу улучшения!

In [26]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0        [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1               [миллиард, газа, год, куб, метр, газпром, добыча, производитель, страна, прогноз]
2                [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                     [ким, зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание]
4                   [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                    [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6               [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7    [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                    [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9                 [участник, рф, организация, государство, область, центр, связь, фсб, март, год]
Name: content_norm, 

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [27]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [28]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [29]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [30]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [31]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [32]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [33]:
keywords[:3]

[['яблоко',
  'активист',
  'акция',
  'дарья',
  'деятельность',
  'молодая человек',
  'политика',
  'виктор',
  'выборы',
  'силовик'],
 ['миллиард куб',
  'куб метр',
  'куб',
  'газпром',
  'газа',
  'миллиард',
  'добыча',
  'метр',
  'добыча газа',
  'холдинг'],
 ['роман',
  'книга',
  'жанр',
  'стихотворение',
  'перевод',
  'читатель',
  'год',
  'поэзия',
  'произведение',
  'том']]

In [34]:
evaluate(data['keywords'], keywords)

Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [69]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [36]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [37]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 36.3 s, sys: 11.4 ms, total: 36.3 s
Wall time: 36.3 s


In [38]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.13
Jaccard -  0.08


In [39]:
keywords_rw.head(10)

0              [яблоко, акция, год, активист, деятельность, свобода, политика, дело, уровень, регион]
1                     [миллиард, газа, куб, год, метр, газпром, добыча, прогноз, месторождение, цена]
2                       [год, книга, стихотворение, роман, мир, идея, читатель, век, перевод, москва]
3                     [ким, зинаида, видео, экспертиза, свидетель, заседание, список, член, год, суд]
4                    [площадь, территория, москва, развитие, га, парк, регион, парка, год, результат]
5                     [гонка, команда, круг, чемпионат, строчка, пилот, бокс, болид, напарник, старт]
6                          [место, рф, позиция, глава, влияние, фрг, россия, новак, участие, рейтинг]
7      [культура, закон, изменение, учреждение, срок, дело, законодательство, первое, акт, концепция]
8                      [сирия, газопровод, год, газа, турция, россия, европа, поток, интерес, проект]
9    [участник, сирия, исламист, территория, прошедшее, организация, житель, округ

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [72]:
import networkx as nx

In [115]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.pagerank(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [116]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 5))

CPU times: user 2min 42s, sys: 867 µs, total: 2min 42s
Wall time: 2min 42s


In [117]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.16
Recall -  0.16
F1 -  0.15
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [44]:
from gensim.summarization import keywords

In [45]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])

In [46]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


Наша имплементация отработала получше.

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

In [10]:
from tqdm import tqdm

In [44]:
# Способ 1. Воспользуемся rutermextract и получим 0,17 с первого же захода

from rutermextract import TermExtractor
term_extractor = TermExtractor()
result = [[result.normalized 
           for result in term_extractor(text,
                                        limit = 5)] 
          for text in tqdm(data['content'])]
evaluate(data['keywords'], result)




  0%|          | 0/999 [00:00<?, ?it/s]


  0%|          | 2/999 [00:00<01:04, 15.37it/s]


  0%|          | 4/999 [00:00<01:03, 15.75it/s]


  1%|          | 6/999 [00:00<01:00, 16.34it/s]


  1%|          | 8/999 [00:00<01:02, 15.89it/s]


  1%|          | 10/999 [00:00<01:17, 12.70it/s]


  1%|          | 12/999 [00:00<01:22, 11.95it/s]


  2%|▏         | 15/999 [00:01<01:08, 14.34it/s]


  2%|▏         | 18/999 [00:01<01:15, 13.00it/s]


  2%|▏         | 21/999 [00:01<01:04, 15.14it/s]


  2%|▏         | 23/999 [00:01<01:00, 16.15it/s]


  3%|▎         | 25/999 [00:01<00:56, 17.13it/s]


  3%|▎         | 27/999 [00:01<01:21, 11.92it/s]


  3%|▎         | 29/999 [00:02<01:14, 12.93it/s]


  3%|▎         | 31/999 [00:02<01:41,  9.55it/s]


  3%|▎         | 33/999 [00:02<01:28, 10.87it/s]


  4%|▎         | 35/999 [00:02<01:24, 11.46it/s]


  4%|▍         | 39/999 [00:02<01:12, 13.33it/s]


  4%|▍         | 41/999 [00:02<01:06, 14.49it/s]


  5%|▍         | 45/999 [00:03<00:54, 17.

 38%|███▊      | 379/999 [00:22<00:30, 20.30it/s]


 38%|███▊      | 382/999 [00:22<00:32, 18.79it/s]


 39%|███▊      | 386/999 [00:23<00:29, 20.85it/s]


 39%|███▉      | 389/999 [00:23<00:32, 18.75it/s]


 39%|███▉      | 392/999 [00:23<00:29, 20.32it/s]


 40%|███▉      | 395/999 [00:23<00:27, 22.01it/s]


 40%|███▉      | 398/999 [00:23<00:28, 21.35it/s]


 40%|████      | 401/999 [00:23<00:26, 22.52it/s]


 41%|████      | 405/999 [00:23<00:24, 24.08it/s]


 41%|████      | 408/999 [00:24<00:24, 24.37it/s]


 41%|████      | 411/999 [00:24<00:23, 24.96it/s]


 41%|████▏     | 414/999 [00:24<00:28, 20.59it/s]


 42%|████▏     | 417/999 [00:24<00:25, 22.55it/s]


 42%|████▏     | 420/999 [00:24<00:27, 20.94it/s]


 42%|████▏     | 423/999 [00:24<00:30, 18.65it/s]


 43%|████▎     | 427/999 [00:24<00:27, 20.91it/s]


 43%|████▎     | 430/999 [00:25<00:27, 20.50it/s]


 43%|████▎     | 433/999 [00:25<00:35, 15.93it/s]


 44%|████▎     | 435/999 [00:25<00:36, 15.65it/s]


 44%|████▎  

 78%|███████▊  | 781/999 [00:44<00:19, 11.05it/s]


 78%|███████▊  | 783/999 [00:44<00:19, 11.01it/s]


 79%|███████▊  | 785/999 [00:45<00:17, 12.49it/s]


 79%|███████▉  | 789/999 [00:45<00:14, 14.70it/s]


 79%|███████▉  | 791/999 [00:45<00:15, 13.11it/s]


 79%|███████▉  | 794/999 [00:45<00:13, 15.76it/s]


 80%|███████▉  | 797/999 [00:45<00:11, 17.57it/s]


 80%|████████  | 800/999 [00:45<00:12, 16.01it/s]


 80%|████████  | 803/999 [00:46<00:12, 15.29it/s]


 81%|████████  | 805/999 [00:46<00:12, 15.96it/s]


 81%|████████  | 807/999 [00:46<00:13, 14.77it/s]


 81%|████████  | 809/999 [00:46<00:15, 12.02it/s]


 81%|████████  | 811/999 [00:46<00:15, 12.43it/s]


 81%|████████▏ | 813/999 [00:46<00:13, 13.97it/s]


 82%|████████▏ | 815/999 [00:47<00:15, 11.72it/s]


 82%|████████▏ | 817/999 [00:47<00:13, 13.34it/s]


 82%|████████▏ | 819/999 [00:47<00:12, 14.64it/s]


 82%|████████▏ | 821/999 [00:47<00:11, 15.90it/s]


 82%|████████▏ | 824/999 [00:47<00:09, 17.60it/s]


 83%|███████

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1


In [58]:
# Способ 2. Попробуем поподбирать оптимальное количество слов, взяты с помощью tf-idf
# Для нашей метрики лучше брать не топ 10, а топ 4
texts_vectors = tfidf.transform(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
for num in range(5, 15):
    print('top {}'.format(num - 1))
    keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-num:-1]]
    evaluate(data['keywords'], keywords)
    print('=======')

top 4
Precision -  0.19
Recall -  0.16
F1 -  0.17
Jaccard -  0.1
top 5
Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1
top 6
Precision -  0.16
Recall -  0.19
F1 -  0.16
Jaccard -  0.1
top 7
Precision -  0.15
Recall -  0.2
F1 -  0.16
Jaccard -  0.1
top 8
Precision -  0.14
Recall -  0.22
F1 -  0.16
Jaccard -  0.09
top 9
Precision -  0.13
Recall -  0.23
F1 -  0.16
Jaccard -  0.09
top 10
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
top 11
Precision -  0.11
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
top 12
Precision -  0.11
Recall -  0.25
F1 -  0.15
Jaccard -  0.08
top 13
Precision -  0.1
Recall -  0.26
F1 -  0.14
Jaccard -  0.08


max_df 0.1
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.2
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.3
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.4
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.5
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.6
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.7
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.8
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09
max_df 0.9
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


In [118]:
# 3 способ. Меняем метрику для центральности в графах
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.eigenvector_centrality_numpy(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 5))
evaluate(data['keywords'], keyword_nx)

Precision -  0.17
Recall -  0.17
F1 -  0.16
Jaccard -  0.1
